In [ ]:
# Импорт библиотек
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


In [ ]:
data = pd.read_csv("Datasets\\sepsis-predict.xls")

In [ ]:
columns_drop = ["Unnamed: 0.1", "Unnamed: 0", "timestamp"]
data.drop(columns_drop, axis=1, inplace=True)

In [ ]:
X = data.drop("SOFA", axis=1).values
y = data["SOFA"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from keras.optimizers import Adam

In [ ]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
ann = Sequential()
ann.add(Dense(units=12, activation="relu", input_dim=X_train.shape[1]))
ann.add(Dense(units=6, activation="relu"))
ann.add(Dense(units=1))

In [ ]:
ann.compile(optimizer=Adam(), loss="mse", metrics=["mae", "mse", "accuracy"])

In [ ]:
class MetricsCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Предсказания модели на обучающем и тестовом наборах данных
        train_pred = self.model.predict(X_train)
        test_pred = self.model.predict(X_test)

        # Вычисление accuracy
        train_acc = accuracy_score(y_train, np.round(train_pred))
        test_acc = accuracy_score(y_test, np.round(test_pred))

        # Вычисление MSE и MAE
        train_mse = mean_squared_error(y_train, train_pred)
        test_mse = mean_squared_error(y_test, test_pred)
        train_mae = mean_absolute_error(y_train, train_pred)
        test_mae = mean_absolute_error(y_test, test_pred)

        # Вывод метрик
        print(
            f'Epoch {epoch + 1}/{self.params["epochs"]}, '
            f'Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}, '
            f'Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}, '
            f'Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}'
        )

In [ ]:
# Обучение модели с использованием созданного обратного вызова
ann.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[MetricsCallback(), EarlyStopping(monitor="val_loss", patience=4)],
)

In [ ]:
ann.evaluate(X_train, y_train)

In [ ]:
ann.evaluate(X_test, y_test)

In [ ]:
# Предсказания на обучающем и тестовом наборах данных
y_train_pred = ann.predict(X_train)
y_test_pred = ann.predict(X_test)

In [ ]:
# Расчет метрик на обучающем наборе
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

In [ ]:
# Расчет метрик на тестовом наборе
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

In [ ]:
# Вывод метрик
print("Метрики на обучающем наборе:")
print("Mean Squared Error (MSE):", train_mse)
print("Mean Absolute Error (MAE):", train_mae)
print("R^2 Score:", train_r2)
print("\nМетрики на тестовом наборе:")
print("Mean Squared Error (MSE):", test_mse)
print("Mean Absolute Error (MAE):", test_mae)
print("R^2 Score:", test_r2)

In [ ]:
predictions = ann.predict(X_test)
predictions_df = pd.DataFrame(np.ravel(predictions), columns=["Predictions"])
comparison_df = pd.concat(
    [pd.DataFrame(y_test, columns=["Real Values"]), predictions_df], axis=1
)
comparison_df

In [ ]:
# Визуализация реальных значений и предсказанных значений
plt.figure(figsize=(10, 6))
plt.plot(
    comparison_df.index, comparison_df["Real Values"], label="Real Values", marker="o"
)
plt.plot(
    comparison_df.index, comparison_df["Predictions"], label="Predictions", marker="x"
)
plt.title("Comparison of Real Values and Predictions")
plt.xlabel("Sample Index")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()